<a href="https://colab.research.google.com/github/MatthewK84/LinkedIn-Learning-Journey/blob/main/Cocktails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests duckdb

In [10]:
import requests
import duckdb
import time

# Function to log messages
def log_message(message):
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {message}")

# Function to fetch data from the Cocktail DB API
def fetch_data():
    url = "https://the-cocktail-db.p.rapidapi.com/filter.php"
    querystring = {"i": "Bourbon"}
    headers = {
        "X-RapidAPI-Key": "Please Get Your Own",
        "X-RapidAPI-Host": "the-cocktail-db.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        log_message(f"Error fetching data: {e}")
        return None

# Function to process data
def process_data(data):
    processed_data = []
    for drink in data['drinks']:
        processed_data.append({
            'id': drink['idDrink'],
            'name': drink['strDrink']
        })
    return processed_data

# Function to store data in DuckDB
def store_data(data, db_file='file.db'):
    try:
        with duckdb.connect(db_file) as con:
            # Create table (if not exists)
            con.execute('''
            CREATE TABLE IF NOT EXISTS bourbon_drinks (
                id TEXT,
                name TEXT
            )
            ''')

            # Insert data
            for item in data:
                con.execute('INSERT INTO bourbon_drinks (id, name) VALUES (?, ?)', (item['id'], item['name']))

            log_message("Bourbon cocktails stored successfully in DuckDB.")
    except Exception as e:
        log_message(f"Error storing data in DuckDB: {e}")

# Main pipeline function
def data_pipeline():
    start_time = time.time()
    log_message("Data pipeline execution started.")

    data = fetch_data()
    if data:
        processed_data = process_data(data)
        store_data(processed_data)
        log_message("Data pipeline execution successful.")
    else:
        log_message("Data pipeline execution failed.")

    end_time = time.time()
    log_message(f"Total execution time: {end_time - start_time} seconds")

def fetch_and_display_data(db_file='file.db'):
    try:
        with duckdb.connect(db_file) as con:
            result = con.execute('SELECT * FROM drinks').fetchall()
            for row in result:
                print(row)
            log_message("Data fetched and displayed successfully.")
    except Exception as e:
        log_message(f"Error fetching data: {e}")

# Run the pipeline
data_pipeline()
fetch_and_display_data()

2023-11-16 21:19:18 - Data pipeline execution started.
2023-11-16 21:19:18 - Bourbon cocktails stored successfully in DuckDB.
2023-11-16 21:19:18 - Data pipeline execution successful.
2023-11-16 21:19:18 - Total execution time: 0.463942289352417 seconds
(13196, 'Long vodka')
(16967, 'Vodka Fizz')
(178362, 'Vodka Slime')
(178363, 'Vodka Lemon')
(178364, 'Vodka Tonic')
(12800, 'Coffee-Vodka')
(14167, 'Vodka Martini')
(15403, 'Vodka Russian')
(12460, 'Vodka And Tonic')
2023-11-16 21:19:18 - Data fetched and displayed successfully.
